In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os
from tqdm import tqdm
import glob

In [2]:
import glob
import os
import pandas as pd

def get_npy_files(root_path):
    # Use a glob pattern to match all .npz files in the directory and subdirectories
    pattern = os.path.join(root_path, '**', '*.npz')
    npy_files = glob.glob(pattern, recursive=True)
    return npy_files

all_npy_files = get_npy_files("/app/dataset/npz_all/npz/layout")+get_npy_files("/app/dataset/npz_all/npz/tile/xla")
print(len(all_npy_files))
# Create a DataFrame from the list of npy files
df = pd.DataFrame(all_npy_files, columns=["path"])
print(len(df))

7868
7868


In [3]:
def find_split(x):
    if "/train" in x:
        return "train"
    elif "/valid" in x:
        return "valid"
    elif "/test" in x:
        return "test"
    return None
def find_configtype(x):
    if "/default" in x:
        return "default"
    elif "/random" in x:
        return "random"
    return "istile"
def find_modeltype(x):
    if "/nlp" in x:
        return "nlp"
    elif "/xla" in x:
        return "xla"
    return None
def find_grapthtype(x):
    if "/layout" in x:
        return "layout"
    elif "/tile" in x:
        return "tile"
    return None
df["split"] = df.path.apply(lambda x : find_split(x))
df["config_type"] = df.path.apply(lambda x : find_configtype(x))
df["modeltype"] = df.path.apply(lambda x : find_modeltype(x))
df["graphtype"] = df.path.apply(lambda x : find_grapthtype(x))


In [4]:
df[df.graphtype=="layout"][["config_type","modeltype","split","path"]].groupby(["modeltype","config_type","split"]).count()

path
modeltype config_type split      
nlp       default     test     17
                      train   198
                      valid    20
          random      test     17
                      train   207
                      valid    20
xla       default     test      8
                      train    61
                      valid     7
          random      test      8
                      train    69
                      valid     7

In [5]:
# num_edges = []
# num_nodes = []
# num_configs = []
# layout_df = df[df.graphtype=="layout"].reset_index(drop=True)
# for d in tqdm(layout_df.path):
#     data = dict(np.load(d))
#     num_edges.append(len(data["edge_index"]))
#     num_nodes.append(len(data["node_feat"]))
#     num_configs.append(len(data["config_runtime"]))
# layout_df["num_edges"] = num_edges
# layout_df["num_nodes"] = num_nodes
# layout_df["num_configs"] = num_configs
# layout_df.to_csv("/app/nn-runtime-network/workdir/csvs/layout_info.csv",index=False)

In [6]:
layout_df = pd.read_csv("/app/nn-runtime-network/workdir/csvs/layout_info.csv")

In [7]:
layout_df = df[df.graphtype=="layout"].merge(on="path",right=layout_df[["path","num_nodes"]])

In [12]:
import os
df = layout_df[(layout_df.num_nodes<30000)|(layout_df.split=="test")]
for spl in df.split.unique():
    if spl!="test":
        continue
    # Replace these with your list of file paths
    file_paths = df[(df.split==spl)&(df.graphtype=="layout")&(layout_df.config_type=="random")&(layout_df.modeltype=="xla")].path.tolist()

    folder_path = f"/app/dataset/various_splits/xla_random/{spl}"
    os.makedirs(folder_path, exist_ok=True)

    # Create symbolic links in the folder
    for file_path in file_paths:
        # Extract the file name from the file path
        file_name = "___".join(file_path.split("/")[-4:])
        
        # Create a symlink in the 'train' folder pointing to the original file
        symlink_path = os.path.join(folder_path, file_name)
        
        # Create the symlink
        os.symlink(file_path, symlink_path)

    print(f'Symlinks to {len(file_paths)} files created in the "{folder_path}" folder.')


Symlinks to 8 files created in the "/app/dataset/various_splits/xla_random/test" folder.


/tmp/ipykernel_1785733/2030166892.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  file_paths = df[(df.split==spl)&(df.graphtype=="layout")&(layout_df.config_type=="random")&(layout_df.modeltype=="xla")].path.tolist()


# All tiles

In [5]:
import os
# df = layout_df
for spl in df.split.unique():
    
    # Replace these with your list of file paths
    if spl=="train" and True:
        file_paths = df[(df.split==spl)&(df.graphtype=="tile")].path.tolist()
    else:
        file_paths = df[(df.split==spl)&(df.graphtype=="tile")].path.tolist()

    folder_path = f"/app/dataset/various_splits/all_tile/{spl}"
    os.makedirs(folder_path, exist_ok=True)

    # Create symbolic links in the folder
    for file_path in file_paths:
        # Extract the file name from the file path
        file_name = "___".join(file_path.split("/")[-4:])
        
        # Create a symlink in the 'train' folder pointing to the original file
        symlink_path = os.path.join(folder_path, file_name)
        
        # Create the symlink
        os.symlink(file_path, symlink_path)

    print(f'Symlinks to {len(file_paths)} files created in the "{folder_path}" folder.')


Symlinks to 844 files created in the "/app/dataset/various_splits/all_tile/test" folder.
Symlinks to 6089 files created in the "/app/dataset/various_splits/all_tile/train" folder.
Symlinks to 296 files created in the "/app/dataset/various_splits/all_tile/valid" folder.
